In [3]:
import yaml
import pandas as pd
from sklearn.ensemble import BaggingRegressor, BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier


In [9]:
with open('config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)
    
train_df = pd.read_csv(cfg['selary']['train_refactor'])     
test_df = pd.read_csv(cfg['selary']['test_refactor'])

train_df = train_df.drop(['Unnamed: 0'], axis =1)
test_df = test_df.drop(['Unnamed: 0'], axis =1)

X_train = train_df.drop(['Credit_Score'], axis=1)
y_train = train_df['Credit_Score']
X_test = test_df.drop(['Credit_Score'], axis=1)
y_test = test_df['Credit_Score']

In [10]:
print(f"Размеры X_train и y_train: {X_train.shape}, {y_train.shape}")
print(f"Размеры X_test и y_test: {X_test.shape}, {y_test.shape}")

Размеры X_train и y_train: (38970, 23), (38970,)
Размеры X_test и y_test: (7737, 23), (7737,)


In [62]:
modelBaggingClass = BaggingClassifier(estimator=LogisticRegression(max_iter=1000), n_estimators=50, random_state=12)
modelBaggingClass.fit(X_train, y_train)

In [81]:
modelBoostingClass = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=5, min_samples_leaf=110), n_estimators=25)
modelBoostingClass.fit(X_train, y_train)

In [15]:
modelGradBoost = GradientBoostingClassifier(max_depth=13, n_estimators=250, learning_rate=0.01, min_samples_leaf=1, max_features=13)
#modelGradBoost.fit(X_train, y_train)

In [13]:
from sklearn.model_selection import GridSearchCV

params = { 'max_depth': range(10,20), 'max_features': range(10,20), 'min_samples_leaf': range(1,5)}

Grad_grid = GridSearchCV(GradientBoostingClassifier(), params, cv=2, verbose=True, n_jobs=4)
pipe_one = make_pipeline(StandardScaler(), Grad_grid)

pipe_one.fit(X_train, y_train)
Grad_grid.best_params_, Grad_grid.best_score_

In [16]:
pipe = make_pipeline(StandardScaler(), modelGradBoost )
pipe.fit(X_train, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(learning_rate=0.01, max_depth=13,
                                            max_features=13,
                                            n_estimators=250))])

In [17]:
y_pred = pipe.predict(X_test)

#средняя абсолютная ошибка
mae = mean_absolute_error(y_test, y_pred)

#среднеквадратичная ошибка
mse = mean_squared_error(y_test, y_pred)

#R^2 score (коэффициент детерминации)
r2 = r2_score(y_test, y_pred)
print(pipe.score(X_test, y_test))
print("MAE:", mae)
print("MSE:", mse)
print("R^2 score:", r2)

0.7876437895825256
MAE: 0.30851751324802895
MSE: 0.5008401189091379
R^2 score: 0.177686976126739
